In [ ]:
####################
import chi
from chi import lease, server, context
from chi.clients import cinder, nova

context.choose_project()
context.choose_site("KVM@TACC")

In [ ]:
volume_name = "MIDAS-block-persist-project-5-GPU"
volume_size = 100  # in GiB
instance_name = "node1-mlops-MIDAS-service-GPU"
mount_path = "/mnt/block"


In [ ]:
cinder_client = chi.clients.cinder()

existing_volumes = cinder_client.volumes.list()

volume = next((v for v in existing_volumes if v.name == volume_name), None)

if volume:
    print(f"Volume '{volume_name}' already exists. Skipping creation.")
else:
    volume = cinder_client.volumes.create(name=volume_name, size=volume_size)
    print(f"Created volume '{volume_name}' with size {volume_size} GiB.")


In [ ]:
target_server = server.get_server(instance_name)

volume_manager = chi.nova().volumes

server_id = target_server.id
volume_id = volume.id

attached = any(
    att.server_id == server_id for att in volume.attachments
)

if attached:
    print(f"Volume already attached to '{instance_name}'.")
else:
    volume_manager.create_server_volume(server_id=server_id, volume_id=volume_id)
    print(f"Volume '{volume_name}' attached to '{instance_name}'.")
